In [1]:
import numpy as np
import pandas as pd

In [2]:
from IPython.display import display, HTML
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate,cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer

## Data Set

<p>
<i><b>Sursă dataset:</b></i>: A fost folosit data setul numit Dermatology care se poate descarca de la adresa <a>http://archive.ics.uci.edu/ml/datasets/dermatology</a>
</p>
<p>
<i><b>Articol relevant:</b></i>: G. Demiroz, H. A. Govenir, and N. Ilter, "Learning Differential Diagnosis of Eryhemato-Squamous Diseases using Voting Feature Intervals", Aritificial Intelligence in Medicine
<p>
<i><b>Scurta decriere:</b></i> Baza de date cuprinde informatii privitoare la conditiile necesare pentru diagnosticarea diferentiata pentru bolile de piele. Cuprinde 366 intrari si 33 atribute ce pot lua valori intre 0 si 3, fiecare dintre acestea indicand severitatea unei conditii medicale. Sunt incluse  6 clase codificate cu cifre intr 1 si 6 dupa cum urmeaza:
</p>
<p>
 <i><b>Cuprinde Clasele:</b></i>
 <ol>               
    <li>psoriasis</li>			    
    <li>seboreic dermatitis  </li>   
    <li>lichen planus</li>          
    <li>pityriasis rosea </li>       
    <li>cronic dermatitis</li>       
    <li>pityriasis rubra pilaris</li>
 </ol>


## Load data

In [3]:
header =['erythema','scaling','definite borders','itching','koebner phenomenon','polygonal papules',
'follicular papules','oral mucosal involvement','knee and elbow involvement','scalp involvement','family history','melanin incontinence',
'eosinophils in the infiltrate','PNL infiltrate','fibrosis of the papillary dermis','exocytosis','acanthosis','hyperkeratosis',
'parakeratosis','clubbing of the rete ridges','elongation of the rete ridges','thinning of the suprapapillary epidermis','spongiform pustule','munro microabcess','focal hypergranulosis',
'disappearance of the granular layer','vacuolisation and damage of basal layer','spongiosis','saw-tooth appearance of retes','follicular horn plug','perifollicular parakeratosis','inflammatory monoluclear inflitrate',
'band-like infiltrate','Age','Class']

print(len(header))
#Alcatuim o lista de stringuri ce ar putea reprezenta missing data in data setul nostru
missingValues = ["n/a", "na", "--"," ","?"]
data_spam = pd.read_csv("./Dataset/dermatology.data",names=header,sep=',',na_values=missingValues)



35


  ###           Se observa existenta unor date lipsa,fiecare coloana ar trebui sa aibe 4601 intrari, acest lucru reiese din informatiile despre setul de date 



In [4]:
display(data_spam.info())
#Afisam numarul de missind data in functie de coloana
print (data_spam.isnull().sum())
#Afisam numarul de missind data in total
print (data_spam.isnull().sum().sum())

#data_spam.replace(to_replace=missingValues, value=np.nan, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   erythema                                  366 non-null    int64  
 1   scaling                                   366 non-null    int64  
 2   definite borders                          366 non-null    int64  
 3   itching                                   366 non-null    int64  
 4   koebner phenomenon                        366 non-null    int64  
 5   polygonal papules                         366 non-null    int64  
 6   follicular papules                        366 non-null    int64  
 7   oral mucosal involvement                  366 non-null    int64  
 8   knee and elbow involvement                366 non-null    int64  
 9   scalp involvement                         366 non-null    int64  
 10  family history                        

None

erythema                                    0
scaling                                     0
definite borders                            0
itching                                     0
koebner phenomenon                          0
polygonal papules                           0
follicular papules                          0
oral mucosal involvement                    0
knee and elbow involvement                  0
scalp involvement                           0
family history                              0
melanin incontinence                        0
eosinophils in the infiltrate               0
PNL infiltrate                              0
fibrosis of the papillary dermis            0
exocytosis                                  0
acanthosis                                  0
hyperkeratosis                              0
parakeratosis                               0
clubbing of the rete ridges                 0
elongation of the rete ridges               0
thinning of the suprapapillary epi

In [5]:
display(HTML("<h3><b>Email Spam Dataset"))
display(HTML(data_spam[5:27].to_html()))


,erythema,scaling,definite borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral mucosal involvement,knee and elbow involvement,scalp involvement,family history,melanin incontinence,eosinophils in the infiltrate,PNL infiltrate,fibrosis of the papillary dermis,exocytosis,acanthosis,hyperkeratosis,parakeratosis,clubbing of the rete ridges,elongation of the rete ridges,thinning of the suprapapillary epidermis,spongiform pustule,munro microabcess,focal hypergranulosis,disappearance of the granular layer,vacuolisation and damage of basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear inflitrate,band-like infiltrate,Age,Class
5,2,3,2,0,0,0,0,0,0,0,0,0,2,1,0,2,2,0,2,0,0,0,1,0,0,0,0,2,0,0,0,1,0,41.0,2
6,2,1,0,2,0,0,0,0,0,0,0,0,0,0,3,1,3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,18.0,5
7,2,2,3,3,3,3,0,2,0,0,0,2,0,0,0,2,3,0,0,0,0,0,0,0,0,2,2,3,2,0,0,3,3,57.0,3
8,2,2,1,0,2,0,0,0,0,0,0,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,2,0,0,0,2,0,22.0,4
9,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,3,2,0,2,0,0,0,0,0,0,0,0,2,0,0,0,2,0,30.0,4
10,3,3,2,1,1,0,0,0,2,2,1,0,0,0,0,0,3,2,3,2,2,2,1,1,0,0,0,0,0,0,0,1,0,20.0,1
11,2,2,0,3,0,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,0,1,0,0,0,0,3,0,0,0,1,0,21.0,2
12,3,3,1,2,0,0,0,0,0,1,0,0,0,2,0,3,1,0,1,0,0,0,0,0,0,0,0,2,0,0,0,1,0,22.0,2
13,2,3,3,0,0,0,0,0,1,1,1,0,0,1,0,0,2,1,2,1,2,3,0,2,0,0,0,0,0,0,0,2,0,10.0,1
14,2,2,3,3,0,3,0,2,0,0,0,2,0,0,0,1,1,1,1,0,0,0,0,0,2,0,3,0,3,0,0,1,3,65.0,3


### Inlocuim datele lipsa 

In [6]:

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
print(imputer)
imputer.fit(data_spam)
data_spam = pd.DataFrame(data=imputer.transform(data_spam), columns=header)



display(HTML("<i>Dataset extract with values imputed:</i>"))
display(HTML(data_spam[5:20].to_html()))

X = data_spam.values[:, :34]
y = data_spam.values[:, -1]

display(X)
display(np.unique(y))

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)


,erythema,scaling,definite borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral mucosal involvement,knee and elbow involvement,scalp involvement,family history,melanin incontinence,eosinophils in the infiltrate,PNL infiltrate,fibrosis of the papillary dermis,exocytosis,acanthosis,hyperkeratosis,parakeratosis,clubbing of the rete ridges,elongation of the rete ridges,thinning of the suprapapillary epidermis,spongiform pustule,munro microabcess,focal hypergranulosis,disappearance of the granular layer,vacuolisation and damage of basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear inflitrate,band-like infiltrate,Age,Class
5,2.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,41.0,2.0
6,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,18.0,5.0
7,2.0,2.0,3.0,3.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0,2.0,0.0,0.0,3.0,3.0,57.0,3.0
8,2.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,22.0,4.0
9,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,30.0,4.0
10,3.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20.0,1.0
11,2.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,21.0,2.0
12,3.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,22.0,2.0
13,2.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,1.0
14,2.0,2.0,3.0,3.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,3.0,0.0,0.0,1.0,3.0,65.0,3.0


array([[ 2.,  2.,  0., ...,  1.,  0., 55.],
       [ 3.,  3.,  3., ...,  1.,  0.,  8.],
       [ 2.,  1.,  2., ...,  2.,  3., 26.],
       ...,
       [ 3.,  2.,  2., ...,  2.,  3., 28.],
       [ 2.,  1.,  3., ...,  2.,  3., 50.],
       [ 3.,  2.,  2., ...,  3.,  0., 35.]])

array([1., 2., 3., 4., 5., 6.])

## Print Function

In [7]:
def print_function(data_set:dict):
    df_print = pd.DataFrame({"Test accuracy for each fold":data_set['test_accuracy'], 
                    "Train accuracy for each fold": data_set['train_accuracy'], 
                    "Average test accuracy %": round(data_set['test_accuracy'].mean() * 100, 4),
                    "Average train accuracy %": round(data_set['train_accuracy'].mean() * 100, 4),
                    "Test F1 score for each fold": data_set['test_f1_macro'],
                    "Train F1 score for each fold": data_set['train_f1_macro'],
                    "Average test F1 score %": round(data_set['test_f1_macro'].mean() * 100, 4),
                    "Average train F1 score %":round(data_set['train_f1_macro'].mean() * 100, 4)
                   })
    display(HTML(df_print.to_html()))

## K-Nearest Neighbors Classifier

In [8]:
# hiperparametri
knn_neighbors = 5
knn_minkowski_p = 3

# scalare date
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# implementare KNN
model = KNeighborsClassifier(n_neighbors=knn_neighbors, p=knn_minkowski_p)
model_cv_stats = cross_validate(model, X_scaled, y, cv=5, scoring=('accuracy', 'f1_macro'), return_train_score=True) 

# statistici
display(HTML(f"<h4>5-fold cross validation for {knn_neighbors}-nearest neighbors classification:</h4>"))
print_function(model_cv_stats)  

,Test accuracy for each fold,Train accuracy for each fold,Average test accuracy %,Average train accuracy %,Test F1 score for each fold,Train F1 score for each fold,Average test F1 score %,Average train F1 score %
0,0.945946,0.982877,95.0833,97.5415,0.946673,0.981964,94.7936,97.3208
1,0.945205,0.976109,95.0833,97.5415,0.942044,0.973730,94.7936,97.3208
2,0.958904,0.972696,95.0833,97.5415,0.953684,0.970766,94.7936,97.3208
3,0.986301,0.976109,95.0833,97.5415,0.988188,0.974982,94.7936,97.3208
4,0.917808,0.969283,95.0833,97.5415,0.909091,0.964597,94.7936,97.3208


## Decision Tree Classifier

In [9]:
# hiperparametri
dt_criterion = 'gini'#default value
dt_splitter = 'best' #intrebarea care reduce cel mai mult incertitudinea
#gini imputity-cantitatea de incertitudine pe un singur nod, cat de amestecate sunt clasificarile din frunze dupa intrebarea din nod
 
# implementare Decision Tree
model = DecisionTreeClassifier(criterion=dt_criterion, splitter=dt_splitter)
model_dc_stats = cross_validate(model, X, y, cv=5, scoring=('accuracy', 'f1_macro'), return_train_score=True)

# statistici
display(HTML(f"<h4>5-fold cross validation for Decision Trees classification:</h4>"))
print_function(model_dc_stats)

,Test accuracy for each fold,Train accuracy for each fold,Average test accuracy %,Average train accuracy %,Test F1 score for each fold,Train F1 score for each fold,Average test F1 score %,Average train F1 score %
0,0.918919,1.0,92.077,100.0,0.895298,1.0,91.1276,100.0
1,0.917808,1.0,92.077,100.0,0.905926,1.0,91.1276,100.0
2,0.904110,1.0,92.077,100.0,0.916287,1.0,91.1276,100.0
3,0.917808,1.0,92.077,100.0,0.899474,1.0,91.1276,100.0
4,0.945205,1.0,92.077,100.0,0.939394,1.0,91.1276,100.0


## Random Forest Classifier

In [10]:
# hiperparametri
rfc_n_estimators = 150
rfc_criterion = 'gini'

# implementare Random Forest
model = RandomForestClassifier(n_estimators=rfc_n_estimators, criterion=rfc_criterion)
model_rfc_stats = cross_validate(model, X, y, cv=5, scoring=('accuracy', 'f1_macro'), return_train_score=True)

display(HTML(f"<h4>5-fold cross validation for Random Forest classification</h4>"))
print_function(model_rfc_stats) 


,Test accuracy for each fold,Train accuracy for each fold,Average test accuracy %,Average train accuracy %,Test F1 score for each fold,Train F1 score for each fold,Average test F1 score %,Average train F1 score %
0,0.972973,1.0,97.2677,100.0,0.974704,1.0,97.0559,100.0
1,1.000000,1.0,97.2677,100.0,1.000000,1.0,97.0559,100.0
2,0.972603,1.0,97.2677,100.0,0.969444,1.0,97.0559,100.0
3,0.986301,1.0,97.2677,100.0,0.984561,1.0,97.0559,100.0
4,0.931507,1.0,97.2677,100.0,0.924086,1.0,97.0559,100.0


## Multilayer Perceptron Classifier

In [13]:
# hiperparametri
mlp_solver = 'adam'
mlp_activation = 'logistic'
mlp_alpha=1e-3
mlp_hidden_layer_sizes = (50,50)
max_iter=1000

# implementare MLP
model = MLPClassifier(solver=mlp_solver, activation=mlp_activation, alpha=mlp_alpha, hidden_layer_sizes=mlp_hidden_layer_sizes, max_iter=max_iter, random_state=0)
model_mpc_stats = cross_validate(model, X, y, cv=5, scoring=('accuracy', 'f1_macro'), return_train_score=True)
print_function(model_mpc_stats)


,Test accuracy for each fold,Train accuracy for each fold,Average test accuracy %,Average train accuracy %,Test F1 score for each fold,Train F1 score for each fold,Average test F1 score %,Average train F1 score %
0,0.959459,1.0,96.9974,100.0,0.956190,1.0,96.6856,100.0
1,1.000000,1.0,96.9974,100.0,1.000000,1.0,96.6856,100.0
2,0.972603,1.0,96.9974,100.0,0.969444,1.0,96.6856,100.0
3,0.986301,1.0,96.9974,100.0,0.984561,1.0,96.6856,100.0
4,0.931507,1.0,96.9974,100.0,0.924086,1.0,96.6856,100.0


## Gaussian Naive Bayes Classifier

In [23]:


# implementare GNB
model = GaussianNB()
model_gnb_stats = cross_validate(model, X, y, cv=5, scoring=('accuracy', 'f1_macro'), return_train_score=True)

# statistici
display(HTML(f"<h4>5-fold cross validation for Gaussian NB classification</h4>"))
print_function(model_gnb_stats)

,Test accuracy for each fold,Train accuracy for each fold,Average test accuracy %,Average train accuracy %,Test F1 score for each fold,Train F1 score for each fold,Average test F1 score %,Average train F1 score %
0,0.905405,0.900685,87.4232,89.9591,0.892797,0.882689,83.8773,88.1502
1,0.849315,0.904437,87.4232,89.9591,0.795756,0.888485,83.8773,88.1502
2,0.849315,0.901024,87.4232,89.9591,0.787987,0.884780,83.8773,88.1502
3,0.876712,0.890785,87.4232,89.9591,0.848276,0.867591,83.8773,88.1502
4,0.890411,0.901024,87.4232,89.9591,0.869048,0.883966,83.8773,88.1502


# Optimizarea Hiperparametrilor

## K-Nearest Neighbors Classifier 

In [24]:
pipe = Pipeline([('scaler', MinMaxScaler()), ('knn', KNeighborsClassifier())])
parameter_grid = {'knn__n_neighbors': list(range(1, 10)), 'knn__p': list(range(1, 5))}
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search =GridSearchCV(pipe, param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)

scores = cross_val_score(grid_search, X, y, cv=strat_k_fold )
print("Scorurile rezultate in urma 5-fold cross validation",scores)
print("Media scorurilor",scores.mean())


grid_search.fit(X, y)
print("Cel mai bun set de parametrii...",grid_search.best_params_)

grid_search = pd.DataFrame(grid_search.cv_results_)
display(HTML(grid_search.to_html()))

Scorurile rezultate in urma 5-fold cross validation [0.95945946 0.98630137 0.98630137 0.94520548 0.94520548]
Media scorurilor 0.9644946316179193
Cel mai bun set de parametrii... {'knn__n_neighbors': 6, 'knn__p': 1}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__n_neighbors,param_knn__p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,0.003747,4.324660e-04,0.008496,4.996674e-04,1,1,"{'knn__n_neighbors': 1, 'knn__p': 1}",0.978261,0.967391,0.945055,0.934066,0.956193,0.017508,11,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.003497,8.660669e-04,0.009744,8.305714e-04,1,2,"{'knn__n_neighbors': 1, 'knn__p': 2}",0.967391,0.978261,0.934066,0.923077,0.950699,0.022792,23,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,0.003748,4.324338e-04,0.035979,7.065507e-04,1,3,"{'knn__n_neighbors': 1, 'knn__p': 3}",0.934783,0.967391,0.923077,0.923077,0.937082,0.018140,30,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.003498,5.000865e-04,0.035729,8.285031e-04,1,4,"{'knn__n_neighbors': 1, 'knn__p': 4}",0.923913,0.967391,0.912088,0.923077,0.931617,0.021175,34,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.003247,4.334647e-04,0.008745,4.330849e-04,2,1,"{'knn__n_neighbors': 2, 'knn__p': 1}",0.945652,0.978261,0.945055,0.945055,0.953506,0.014294,14,0.985401,0.970803,0.989091,0.992727,0.984506,0.008324
5,0.003247,4.333955e-04,0.009244,4.341525e-04,2,2,"{'knn__n_neighbors': 2, 'knn__p': 2}",0.956522,0.989130,0.934066,0.934066,0.953446,0.022550,16,0.985401,0.974453,0.981818,0.992727,0.983600,0.006584
6,0.003498,4.996661e-04,0.038977,2.912934e-03,2,3,"{'knn__n_neighbors': 2, 'knn__p': 3}",0.934783,0.978261,0.912088,0.923077,0.937052,0.025109,31,0.985401,0.974453,0.981818,0.992727,0.983600,0.006584
7,0.003748,4.325695e-04,0.036228,8.282350e-04,2,4,"{'knn__n_neighbors': 2, 'knn__p': 4}",0.934783,0.956522,0.923077,0.923077,0.934365,0.013656,33,0.985401,0.974453,0.981818,0.985455,0.981782,0.004481
8,0.003247,4.336032e-04,0.008996,9.739479e-07,3,1,"{'knn__n_neighbors': 3, 'knn__p': 1}",0.978261,0.956522,0.978022,0.934066,0.961718,0.018242,5,0.978102,0.981752,0.978182,0.992727,0.982691,0.005979
9,0.003247,4.338763e-04,0.008995,8.344650e-07,3,2,"{'knn__n_neighbors': 3, 'knn__p': 2}",0.978261,0.978261,0.934066,0.934066,0.956163,0.022097,13,0.974453,0.981752,0.978182,0.992727,0.981778,0.006828


## Decision Tree Classifier

In [25]:
pipe = Pipeline([('dtc', DecisionTreeClassifier())])

parameter_grid = { 'dtc__criterion': ['gini', 'entropy'],'dtc__splitter': ['best', 'random'] }
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search =GridSearchCV(pipe, param_grid=parameter_grid, scoring='accuracy', cv=4)
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
scores = cross_val_score(grid_search, X, y, cv=strat_k_fold )

print("Scorurile rezultate in urma 5-fold cross validation",scores)
print("Media scorurilor",scores.mean())
grid_search.fit(X, y)
print("Cel mai bun set de parametrii...",grid_search.best_params_)
grid_search = pd.DataFrame(grid_search.cv_results_)
display(HTML(grid_search.to_html()))


Scorurile rezultate in urma 5-fold cross validation [0.93243243 0.95890411 0.93150685 0.93150685 0.95890411]
Media scorurilor 0.9426508700481303
Cel mai bun set de parametrii... {'dtc__criterion': 'gini', 'dtc__splitter': 'best'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_dtc__criterion,param_dtc__splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001999,0.000002,0.00025,0.000433,gini,best,"{'dtc__criterion': 'gini', 'dtc__splitter': 'best'}",0.913043,0.913043,0.923077,0.956044,0.926302,0.017653,1
1,0.001249,0.000432,0.00075,0.000433,gini,random,"{'dtc__criterion': 'gini', 'dtc__splitter': 'random'}",0.913043,0.902174,0.923077,0.945055,0.920837,0.015816,2
2,0.001748,0.000432,0.00050,0.000500,entropy,best,"{'dtc__criterion': 'entropy', 'dtc__splitter': 'best'}",0.847826,0.934783,0.945055,0.945055,0.918180,0.040835,3
3,0.001999,0.000002,0.00000,0.000000,entropy,random,"{'dtc__criterion': 'entropy', 'dtc__splitter': 'random'}",0.902174,0.858696,0.912088,0.945055,0.904503,0.030845,4


## Random Forest Classifier

In [26]:
pipe = Pipeline([('rfc', RandomForestClassifier())])

parameter_grid = { 'rfc__criterion': ['gini', 'entropy'],'rfc__n_estimators': np.linspace(start=1, stop=150, num=25, dtype=int)}
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search =GridSearchCV(pipe, param_grid=parameter_grid, scoring='accuracy', cv=4)
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
scores = cross_val_score(grid_search, X, y, cv=strat_k_fold )

print("Scorurile rezultate in urma 5-fold cross validation",scores)
print("Media scorurilor",scores.mean())
grid_search.fit(X, y)
print("Cel mai bun set de parametrii...",grid_search.best_params_)
grid_search = pd.DataFrame(grid_search.cv_results_)
display(HTML(grid_search.to_html()))

#Do not run this it can take a while

Scorurile rezultate in urma 5-fold cross validation [0.95945946 0.97260274 0.98630137 1.         0.97260274]
Media scorurilor 0.9781932617549056
Cel mai bun set de parametrii... {'rfc__criterion': 'gini', 'rfc__n_estimators': 118}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__criterion,param_rfc__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003498,0.000499,0.000999,1.072884e-06,gini,1,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 1}",0.815217,0.836957,0.923077,1.000000,0.893813,0.073386,49
1,0.019489,0.002596,0.001998,9.884312e-07,gini,7,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 7}",0.934783,0.989130,0.956044,0.934066,0.953506,0.022383,46
2,0.036727,0.008223,0.003749,4.321562e-04,gini,13,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 13}",0.956522,1.000000,0.945055,0.956044,0.964405,0.021056,40
3,0.045472,0.000500,0.003998,1.415532e-06,gini,19,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 19}",0.945652,0.978261,0.945055,0.934066,0.950758,0.016535,47
4,0.060963,0.001869,0.004748,4.330516e-04,gini,25,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 25}",0.967391,1.000000,0.956044,0.956044,0.969870,0.018002,25
5,0.079452,0.001500,0.006245,4.332606e-04,gini,32,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 32}",0.967391,0.967391,0.923077,0.967033,0.956223,0.019138,45
6,0.090445,0.000499,0.006996,7.066351e-04,gini,38,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 38}",0.978261,0.989130,0.956044,0.945055,0.967123,0.017451,35
7,0.106187,0.002771,0.009742,3.028633e-03,gini,44,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 44}",0.967391,1.000000,0.956044,0.967033,0.972617,0.016454,15
8,0.121177,0.004490,0.008993,1.025477e-06,gini,50,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 50}",0.978261,1.000000,0.956044,0.945055,0.969840,0.021126,27
9,0.135917,0.005193,0.011493,2.059322e-03,gini,56,"{'rfc__criterion': 'gini', 'rfc__n_estimators': 56}",0.978261,1.000000,0.945055,0.945055,0.967093,0.023340,37


## Gaussian Naive Bayes Classifier

In [27]:
pipe = Pipeline([('gnb', GaussianNB())])

parameter_grid = { 'gnb__var_smoothing': np.linspace(start=1e-9, stop=1e-2, num=100)}
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search =GridSearchCV(pipe, param_grid=parameter_grid, scoring='accuracy', cv=4)
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
scores = cross_val_score(grid_search, X, y, cv=strat_k_fold )

print("Scorurile rezultate in urma 5-fold cross validation",scores)
print("Media scorurilor",scores.mean())
grid_search.fit(X, y)
print("Cel mai bun set de parametrii...",grid_search.best_params_)
grid_search = pd.DataFrame(grid_search.cv_results_)
display(HTML(grid_search.to_html()))

Scorurile rezultate in urma 5-fold cross validation [0.90540541 0.97260274 1.         0.98630137 0.97260274]
Media scorurilor 0.9673824509440948
Cel mai bun set de parametrii... {'gnb__var_smoothing': 0.0004040413636363637}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gnb__var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001749,4.328782e-04,0.001249,4.326733e-04,1e-09,{'gnb__var_smoothing': 1e-09},0.913043,0.869565,0.857143,0.901099,0.885213,0.022692,100
1,0.001500,5.003223e-04,0.001498,4.991299e-04,0.000101011,{'gnb__var_smoothing': 0.00010101109090909092},0.967391,0.978261,0.923077,0.956044,0.956193,0.020671,30
2,0.001999,1.148067e-06,0.000999,1.198039e-06,0.000202021,{'gnb__var_smoothing': 0.00020202118181818183},0.978261,0.989130,0.934066,0.956044,0.964375,0.021176,8
3,0.002248,4.337742e-04,0.001249,4.316408e-04,0.000303031,{'gnb__var_smoothing': 0.0003030312727272728},0.978261,0.989130,0.934066,0.956044,0.964375,0.021176,8
4,0.001998,8.980345e-07,0.001251,4.336014e-04,0.000404041,{'gnb__var_smoothing': 0.0004040413636363637},0.989130,0.989130,0.934066,0.956044,0.967093,0.023368,1
5,0.001749,4.315704e-04,0.001249,4.330166e-04,0.000505051,{'gnb__var_smoothing': 0.0005050514545454546},0.978261,0.978261,0.934066,0.956044,0.961658,0.018331,10
6,0.002000,1.184619e-06,0.001499,4.991890e-04,0.000606062,{'gnb__var_smoothing': 0.0006060615454545455},0.978261,0.978261,0.934066,0.956044,0.961658,0.018331,10
7,0.002001,1.203955e-06,0.000999,5.057622e-07,0.000707072,{'gnb__var_smoothing': 0.0007070716363636365},0.978261,0.978261,0.934066,0.967033,0.964405,0.018106,2
8,0.002001,1.110312e-06,0.001248,4.308823e-04,0.000808082,{'gnb__var_smoothing': 0.0008080817272727274},0.978261,0.978261,0.934066,0.967033,0.964405,0.018106,2
9,0.001999,1.168008e-06,0.001248,4.319505e-04,0.000909092,{'gnb__var_smoothing': 0.0009090918181818183},0.978261,0.978261,0.934066,0.956044,0.961658,0.018331,10


## Multilayer Perceptron Classifier

In [ ]:
pipe = Pipeline([('mlp', MLPClassifier(max_iter= 1000))])

parameter_grid = { 'mlp__alpha': np.linspace(start=0, stop=1e-1, num=50),
                  'mlp__activation': ['identity', 'logistic', 'tanh', 'relu']
                 }

strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search =GridSearchCV(pipe, param_grid=parameter_grid, scoring='accuracy', cv=4)
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
scores = cross_val_score(grid_search, X, y, cv=strat_k_fold )

print("Scorurile rezultate in urma 5-fold cross validation",scores)
print("Media scorurilor",scores.mean())
grid_search.fit(X, y)
print("Cel mai bun set de parametrii...",grid_search.best_params_)
grid_search = pd.DataFrame(grid_search.cv_results_)
display(HTML(grid_search.to_html()))

#Do not run this it can take a while

C:\Anaconda3\envs\machine-learning\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Anaconda3\envs\machine-learning\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
